In [51]:
# Import Python Tools:
from IPython.display import display, Latex
import pandas as pd
import numpy as np
from numpy import sqrt, pi, average, cos, sin, tan, arcsin, arccos, arctan, deg2rad, rad2deg
# import sectionproperties as shape

from ASCE import load_combos

from steel_design.AISC import aisc
steel_density = 490 / 12**3 #pci

from concrete_design import material_properties as conc
from concrete_design.steel_reinforcement import rebar_df as rebar
conc_density = 150.0 #pcf

from masonry_design import material_properties as cmu
from masonry_design.material_properties import NW_CMU #psf

from wood_design.sawn_lumber.ref_design_values.table_4a import table_4a_load
from wood_design.sawn_lumber.ref_design_values.C_F import C_F_load
table_4a = table_4a_load(species="DF")

<div style="page-break-after: always;"></div>

# Concrete Beam Design
## Inputs

In [52]:
# Materials
fc = 6000 #psi
fy = 60000 #psi
fyt = 60000 #psi
Es = 29000000 #psi

# Dimensions
b = 15 #in
h = 24 #in
ln = 20*12 #in
cover_t = 1.5 #in
cover_b = 1.5 #in

#### Deep beam check

- span does not exceed 4 * h

- concentrated load exists within a distance 2h from face of support

In [53]:
if ln < 4*h:
    print("DEEP BEAM")
else:
    print("beam theory OK")

beam theory OK


### Loads

In [54]:
SW = conc_density * b * h / 12**2 #plf
DL = 15 #psf
LL = 40 #psf

LN = ln / 12 #ft
trib = 15 #ft
D = DL * trib / 12 + SW #plf
L = LL * trib / 12 #plf
w_u = load_combos.LRFD(D, L) #plf

Mu = w_u * LN**2 / 8 / 1000 #kip-ft
Vu = w_u * LN / 2 / 1000 #kips
print(f"Mu = {round(Mu, 2)} kip-ft")
print(f"Vu = {round(Vu, 2)} kips")

Mu = 27.62 kip-ft
Vu = 5.53 kips


### Materials and Geometry

In [55]:
# Shear Steel
stirrup_size = "#3"
no_legs = 2
s = 12 #in O.C.

db_tie = rebar.loc[stirrup_size, "Diameter"]
Av = no_legs * rebar.loc[stirrup_size, "Area"]
f"Av = {round(Av, 2)} in^2"

'Av = 0.22 in^2'

In [56]:
# Tension Steel
bar = "#11"
Ab_t = rebar.loc[bar, "Area"]
db_t = rebar.loc[bar, "Diameter"]
n = 6 #number of bars
layers = 1

d_t = h - cover_b - db_tie - db_t/2
print(f"d_t = {round(d_t, 3)} in")

As_t = n * Ab_t #in^2
print(f"As = {round(As_t, 3)} in^2")

d_t = 21.42 in
As = 9.36 in^2


In [57]:
# Compression Steel
bar = "#9"
Ab_c = rebar.loc[bar, "Area"]
db_c = rebar.loc[bar, "Diameter"]
n = 3 #number of bars
layers = 1

d_c = cover_t + db_tie + db_c/2
print(f"d_c = {round(d_c, 3)} in")

As_c = n * Ab_c #compression reinforcement
print(f"As = {round(As_c, 3)} in^2")

d_c = 2.439 in
As = 3.0 in^2


## Strength Design

### $\phi$ Mn $\geq$ Mu

In [58]:
include_compression_steel = "Yes"
if include_compression_steel == "No":
    As_c = 0

print(As_c)

3.0


#### $\phi$ Factor Calculation

In [59]:
c = bm.compression_block(As_t, fy, Es, b, fc, Apr_s=As_t, dpr=d_c) 
a = bm.equiv_compr_block(c)
eps_t = bm.tension_steel_strain(d_t, c)
eps_cs = bm.compr_steel_strain(c, eps_t, d_t)
φ_b = bm.phi_factor(eps_t, fy, Es)

c, a, eps_t, eps_cs, φ_b

(4.079919100602714, 3.4679312355123066, 0.01275031230165987, 0.003, 0.9)

#### Flexural Strength

In [60]:
fs = min(eps_cs * Es, fy) #psi (negative strain)

C_s = As_c * fs #lbs
C_c = As_t*fy - C_s #lbs
T = As_t * fy #lbs

Mn = T*d_t - C_c*a/2 - C_s*d_c #lb-in
φMn = φ_b * Mn

print(f"φ_b = {round(φ_b, 3)}")
print(f"Mu = {round(Mu, 2)} kip-ft")
print(f"Mn = {round(Mn / 12 / 1000, 2)} kip-ft")
print(f"φMn = {round(φMn / 12 / 1000, 2)} kip-ft")
f"DCR = {round(Mu / (φMn / 12 / 1000), 3)}"

φ_b = 0.9
Mu = 27.62 kip-ft
Mn = 910.73 kip-ft
φMn = 819.66 kip-ft


'DCR = 0.034'

In [61]:
Mn /1000, C_s/1000, C_c/1000

(10928.770720264254, 180.0, 381.6)

### $\phi$ Vn $\geq$ Vu

#### Shear Strength

In [62]:
Vc = 2 * sqrt(fc) * b * d_t #lbs
f"Vc = {round(Vc, 2)} lbs"

'Vc = 49775.58 lbs'

In [63]:
Vs = Av * fyt * d_t / s #lbs
f"Vs = {round(Vs, 2)} lbs"

'Vs = 23562.0 lbs'

In [64]:
φ_v = 0.75
Vn = Vc + Vs
print(f"Vn = {round(Vn, 2)} lbs")
print(f"φVn = {round(φ_v * Vn, 2)} lbs")

Vn = 73337.58 lbs
φVn = 55003.19 lbs


In [65]:
f"DCR = {round(Vu / (φ_v * Vn / 1000), 3)}"

'DCR = 0.1'

### $\phi$ Tn $\geq$ Tu

### $\phi$ Pn $\geq$ Pu

## Deflection

### Short-term Deflection

In [66]:
w_a = load_combos.ASD(D, L)

Ma = w_a * LN**2 / 8 * 12 #lb-in
f"Ma = {round(Ma / 12, 2)} lb-ft"

'Ma = 22187.5 lb-ft'

In [67]:
fr = 7.5 * sqrt(fc) #psi
Ig = b * h**3 / 12 #in^4
print(f"Ig = {round(Ig, 2)} in")
yt = h / 2 #in
Mcr = fr * Ig / yt #lb-in
f"Mcr = {round(Mcr / 12, 2)} lb-ft"

Ig = 17280.0 in


'Mcr = 69713.7 lb-ft'

In [68]:
# Elastic Modulus
Ec = elastic_modulus(fc=fc) #psi
print(f"Ec = {round(Ec, 2)} ksi")
# Moment of Inertia
Ie = Ig
Icr = cracked_moment_inertia(b=b, d=d_t, Ec=Ec, As=As_t)
# Icr = cracked_moment_inertia(b=b, h=h, d=d_t, d_c=d_c, Ec=Ec, As=As_t, As_c=As_c)
print(f"Icr = {round(Icr, 2)} in^4")
if Ma > 2/3*Mcr:
    Ie = Icr / (1 - ((2/3)*Mcr/Ma)**2 * (1 - Icr/Ig))
    print(f"Ie = {round(Ie, 2)} in")
else:
    print("section uncracked")

#Elastic Deflection
defl_elastic = 5 * (w_a/12) * ln**4 / (384 * Ec * 1000 * Ie)
f"Elastic Deflection = {round(defl_elastic, 3)} in"

Ec = 4695982.33 ksi
Icr = 14852.21 in^4
section uncracked


'Elastic Deflection = 0.0 in'

### Long-term Deflection

In [69]:
# Deflection Factor
xi = 2.0 #duration factor
rho_c = As_c / (b * d_c)

lmbda_dlt = xi / (1 + 50*rho_c)
f"lmbda_dlt = {round(lmbda_dlt, 3)}"

'lmbda_dlt = 0.392'

In [70]:
# Long-Term Deflection
defl_long = defl_elastic * lmbda_dlt #in
f"Long-Term Deflection = {round(defl_long, 3)} in"

'Long-Term Deflection = 0.0 in'

In [71]:
# Total Deflection
defl_total = defl_elastic + defl_long
f"Total Deflection = {round(defl_total, 3)} in"

'Total Deflection = 0.0 in'

## Minimum Requirements

### Dimension Requirements

#### Minimum Beam Depth

In [72]:
h_min = ln / 16
print(f"h_min (simply supported) = {round(h_min, 2)} in")

h_min = ln / 18.5
print(f"h_min (one end continuous) = {round(h_min, 2)} in")

h_min = ln / 21
print(f"h_min (both ends continuous) = {round(h_min, 2)} in")

h_min = ln / 8
print(f"h_min (cantilever) = {round(h_min, 2)} in")

h_min (simply supported) = 15.0 in
h_min (one end continuous) = 12.97 in
h_min (both ends continuous) = 11.43 in
h_min (cantilever) = 30.0 in


In [73]:
Vu <= φ_v * (Vc + 8*sqrt(fc) * b * d_t)

True

### Minimum Reinforcement

#### Flexural Reinforcement

In [74]:
As_min = max(3*sqrt(fc)/fy, 300/fy) * b * d_t #in^2

f"As,min = {round(As_min, 3)} in^2"

'As,min = 1.606 in^2'

#### Shear Reinforcement

In [75]:
if Vu > φ_v*sqrt(fc)*b*d_t:
    Av_min = max(0.75*sqrt(fc), 50) * b / fyt #in^2
    print(f"Av,min = {round(Av_min, 3)} in^2")
else:
    Av_min = 0
    print("Shear reinforcement not required")

Shear reinforcement not required


#### Torsional Reinforcement